In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from my_functions import *

In [ ]:
tcurves = load_tcurves(load_filter_tags())
w_central = central_wavelength(tcurves)

In [ ]:
## Load gal catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Gal_jpas_mock_flam_1sq.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

gal_flx = []
gal_err = []
zspec = []
with open(filename) as f:
    rdr = csv.reader(f, delimiter=' ')
    next(rdr, None)
    for row in rdr:
        gal_flx.append(row[2 : 2 + 60])
        gal_err.append(row[2 + 60 : 2 + 60 + 60])
        zspec.append(row[-1])
gal_flx = np.array(gal_flx).T[my_filter_order].astype(float)
gal_err = np.array(gal_err).T[my_filter_order].astype(float)
zspec = np.array(zspec).astype(float)

In [ ]:
N_sources = gal_flx.shape[1]
N_nb = 5
ew0min = 30

In [ ]:
nb_c_min = 6
nb_c_max = 54
line_gal = np.zeros((nb_c_max - nb_c_min, N_sources)).astype(bool)
cont_est_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
cont_err_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
i = 0
for nb_c in range(nb_c_min, nb_c_max):
    z_nb = w_central[nb_c] / 1215.67 - 1
    fwhm = nb_fwhm(nb_c, tcurves=tcurves)
    cont_est_gal, cont_err_gal = stack_estimation(gal_flx, gal_err, nb_c, N_nb, False)
    line_gal[i] = (
        (gal_flx[nb_c] - cont_est_gal > 3 * (cont_err_gal**2 + gal_err[nb_c]**2) ** 0.5)
        & (gal_flx[nb_c] - cont_est_gal > ew0min * (1 + z_nb) * cont_est_gal / fwhm)
    )
    cont_est_Arr[i] = cont_est_gal
    cont_err_Arr[i] = cont_err_gal
    i += 1

In [ ]:
j = 0
for mock_n in np.where(zspec > 0)[0]:
    try:
        first = np.where(line_gal[:, mock_n])[0][0] + nb_c_min
    except:
        continue

    fig = plt.figure(figsize=(10,8))
    ax = plot_JPAS_source(gal_flx[:, mock_n], gal_err[:, mock_n])

    for nb in np.where(line_gal[:, mock_n])[0]:
        idx = nb + nb_c_min
        ax.errorbar(w_central[idx], cont_est_Arr[nb, mock_n], fmt='^', c='C4',
            markersize=10, yerr=cont_err_Arr[nb, mock_n], capsize=3,
            capthick=3, elinewidth=4)
    plt.show()

    z_nb = w_central[first] / 1215.67 - 1

    print('z spec = {0:0.2f}'.format(zspec[mock_n]))
    print('z NB   = {0:0.2f}'.format(z_nb))

    j += 1
    if j == 10: break

In [ ]:
N_sources

In [ ]:
# Arr where the center of the Lya line is. -1 means no lines detected
line_lya_center = np.ones(N_sources, dtype=int) * -1

for mock_n in range(N_sources):
    try:
        first = np.where(line_gal[:, mock_n])[0][0] # First True in line_gal
    except:
        continue
    if first > 40: # We don't expect lines past here
        continue
    if flux_to_mag(gal_flx[-2, mock_n], 5000) < 23: continue

    line_nb_width = 0
    while line_gal[first + line_nb_width + 1, mock_n]:
        line_nb_width += 1

    line_lya_center[mock_n] = np.argmax(
        gal_flx[first + nb_c_min : first + nb_c_min + line_nb_width + 1, mock_n]
    ) + first + nb_c_min
print('Src with a line: {}'.format(len(np.where((line_lya_center != -1))[0])))

n_lines = []
for mock_n in range(N_sources):
    n_lines.append(len(np.where(line_gal[:, mock_n])[0]))
print('Src with single line: {}'.format(len(np.where(np.array(n_lines) == 1)[0])))
print('Src with single line and zspec > 2.5: {}'.format(
    len(np.where((np.array(n_lines) == 1) & (zspec > 2.5))[0]))
)

In [ ]:
np.max(zspec)